In [1]:
import os
import pandas as pd

In [2]:
DATA_DIR='../../../docs/metrics/events/promote/_data/'
os.makedirs(DATA_DIR, exist_ok=True)

In [3]:
data = pd.read_csv('../../../data/metrics/events/promote-events/events.csv', parse_dates=['time'])

In [4]:
pd.Series({
    'total_events': len(data.id.unique()),
    'total_event_instances': len(data.time),
    'earliest_event': data.time.min(),
    'latest_event': data.time.max(),
}).to_json(
    os.path.join(DATA_DIR, 'headlines.json'),
    date_format='iso', indent=2
)

In [5]:
events = pd.DataFrame({
    'events': data.groupby('id').agg({ 'time': 'min' }).reset_index().groupby('time').id.count(),
    'event_instances': data.groupby('time').id.count(),
}).fillna(0).astype(int)

In [6]:
def cumulate(data):
    return pd.concat([
        data,
        data.cumsum().rename(columns=lambda c: f'cumulative_{c}')
    ], axis=1)

In [7]:
events.resample('W-FRI').sum().pipe(cumulate).to_csv(os.path.join(DATA_DIR, 'by_week.csv'))

In [8]:
events.resample('MS').sum().pipe(cumulate).to_csv(os.path.join(DATA_DIR, 'by_month.csv'))